# Custom data loader

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset

In [2]:
class DiabetesDataset(torch.utils.data.Dataset):

    def __init__(self):
        super(DiabetesDataset, self).__init__()
        xy = np.loadtxt("data/diabetes.csv", delimiter=",", dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [3]:
dataset = DiabetesDataset()

In [6]:
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

## Model definition


In [9]:
class DiabetesPredictionModel(nn.Module):

    def __init__(self):
        super(DiabetesPredictionModel, self).__init__()

        self.l1 = nn.Linear(8, 6)
        self.l2 = nn.Linear(6, 4)
        self.l3 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # NN architecture
        # x --- l1 --- sigmoid --- l2 --- sigmoid --- l3 --- sigmoid --- y_hat

        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))

        return y_pred

In [11]:
diabetes_prediction_model = DiabetesPredictionModel()

## Loss and Optimiser

In [12]:
criterion = nn.BCELoss(size_average=True)

In [14]:
learning_rate = 0.1
optimiser = optim.SGD(diabetes_prediction_model.parameters(), lr=learning_rate)

## Training

In [16]:
for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)

        y_pred = diabetes_prediction_model(inputs)

        loss = criterion(y_pred, labels)

        print(epoch, i, loss.data.item())

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()


0 0 0.842182993888855
0 1 0.7863770127296448
0 2 0.7568399310112
0 3 0.7208936810493469
0 4 0.7097887396812439
0 5 0.7176643013954163
0 6 0.7116217613220215
0 7 0.7059764862060547
0 8 0.7029955387115479
0 9 0.6965780258178711
0 10 0.6924525499343872
0 11 0.6895150542259216
0 12 0.6830314993858337
0 13 0.6765130758285522
0 14 0.7078114151954651
0 15 0.6781600713729858
0 16 0.6746430397033691
0 17 0.6710585355758667
0 18 0.66220623254776
0 19 0.6443168520927429
0 20 0.6605961918830872
0 21 0.6408753395080566
0 22 0.6559908986091614
0 23 0.6117717027664185
1 0 0.591687798500061
1 1 0.6214637756347656
1 2 0.6909410953521729
1 3 0.6897903680801392
1 4 0.7170020341873169
1 5 0.5983550548553467
1 6 0.732850968837738
1 7 0.6757897138595581
1 8 0.6105575561523438
1 9 0.6621018648147583
1 10 0.6619400382041931
1 11 0.6191132664680481
1 12 0.5860544443130493
1 13 0.5961658358573914
1 14 0.6271569132804871
1 15 0.6793193817138672
1 16 0.6274148225784302
1 17 0.6972355842590332
1 18 0.5933396220207

## Further work
* https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel